## 1. 모듈 및 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece==0.1.85
!pip install transformers==2.1.1
!pip install torch
!pip install emoji
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

     |████████████████████████████████| 356.7 MB 29 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595744 sha256=b09ab7884e37d46ac189f71fc136b9194e8d27033404fddf9014f94f95fb3fc8
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 895 kB 35.1 MB/s 
     |████████████████████████████████| 131 kB 44.2 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 7.8 MB 32.8 MB/s 
     |████████████████████████████████| 138 kB 46.8 MB/s 
     |████████████

In [ ]:
import pandas as pd

happy = pd.read_excel('/content/drive/My Drive/data/행복.xlsx')
fun = pd.read_excel('/content/drive/My Drive/data/즐거움.xlsx')
sad = pd.read_excel('/content/drive/My Drive/data/슬픔.xlsx')
anger = pd.read_excel('/content/drive/My Drive/data/분노.xlsx')


happy

,Unnamed: 0,keyWorld,raw_text
0,0.0,해피,히히 다정둥이 바바님🥰 바바님 텐션이 좋으셔서 항상 기분 좋아요! 해피바이러스 바...
1,1.0,해피,나 내 생일날 해피가랜드 입고싶은데 그러려면 사야함(칰쇼)
2,2.0,해피,사랑을 하고 있어... 지윤이 먹어주면 지영이 대박 해피함
3,3.0,해피,미첫내...? 방학숙제 많이 없는 사람 존나 해피함
4,4.0,해피,찾기 어려운것도 있지만 한번 사면 재구매같은걸로 꾸준히 사기도 편하고 빨리 오고 ...
...,...,...,...
5571,NaN,NaN,완내스
5572,NaN,NaN,졌잘싸
5573,NaN,NaN,존맛
5574,NaN,NaN,존맛탱


In [ ]:
import emoji

def extract_emojis(s):
  return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

def preprocessing(data, label):
  import re
  dt = data['raw_text'].copy() #문장만 선택
  dt = dt.dropna() #결측치 제거
  dt = dt.drop_duplicates() #중복 제거
  sentences = dt.tolist()
  new_sent=[]
  for i in range(len(sentences)):
    sent = sentences[i] 
    emo=extract_emojis(sent)
    # print(sent)
    for k in emo:
      sent = re.sub(k,' ',sent)
    if type(sent) != str: # 문장 중 str 아닌 것 처리
      sent = str(sent)
    if len(sent) < 2: continue #길이 1 이상인 것만 선택
    sent = re.sub('\n',' ',sent)
    sent = re.sub('ㅋㅋ+','ㅋㅋ',sent)
    sent = re.sub('ㅠㅠ+','ㅠㅠ',sent)
    sent = re.sub('ㅇㅇ+','ㅇㅇ',sent) 
    sent = re.sub('ㄷㄷ+','ㄷㄷ',sent) 
    sent = re.sub('ㅎㅎ+','ㅎㅎ',sent)
    sent = re.sub('ㅂㅂ+','ㅂㅂ',sent) 
    sent = re.sub(';;;+',';;',sent)
    sent = re.sub('!!!+','!!',sent)
    sent = re.sub('~+','~',sent)
    sent = re.sub('[?][?][?]+','??',sent) 
    sent = re.sub('[.][.][.]+','...',sent) 
    sent = re.sub('[-=+,#/:$@*\"※&%ㆍ』\\‘|\(\)\[\]\<\>`\'…》]','',sent)
    new_sent.append(sent)
  dt = pd.DataFrame(pd.Series(new_sent), columns=['raw_text'])
  dt['emotion'] = label
  return dt

In [ ]:
happy = preprocessing(happy, '행복')
fun = preprocessing(fun, '즐거움')
sad = preprocessing(sad, '슬픔')
anger = preprocessing(anger, '분노')

for i in [happy, fun, sad, anger]:
  print('레이블', i['emotion'][0],len(i))



레이블 행복 5413
레이블 즐거움 5301
레이블 슬픔 5711
레이블 분노 4382


In [ ]:


# 각 감정별 키워드 데이터가 약 10개 씩으로 이루어져 있기 때문에 마지막 키워드에 대한 데이터 1000개를 평가 데이터로 선택
sentence_train = pd.concat([happy, fun, sad, anger], axis=0, ignore_index=True)


for i in ['행복','즐거움','슬픔','분노']:
  print('sentence_train',i,len(sentence_train[sentence_train['emotion'] == i]))


sentence_train 행복 5413
sentence_train 즐거움 5301
sentence_train 슬픔 5711
sentence_train 분노 4382


In [ ]:
def label(x):
  if x=='행복': return 0.0
  elif x=='즐거움': return 1.0
  elif x=='슬픔': return 2.0
  elif x=='분노': return 3.0
  else: return x

sentence_train["emotion"] = sentence_train["emotion"].apply(label)


dtls = [list(sentence_train.iloc[i,:]) for i in range(len(sentence_train))]
dtls[:5] 

[[' 히히 다정둥이 바바님  바바님 텐션이 좋으셔서 항상 기분 좋아요! 해피바이러스 바바님   바바님과 함께라면 언제나 꽃길! 바바님 행복한 하루 보내시기 ',
  0.0],
 ['나 내 생일날 해피가랜드 입고싶은데 그러려면 사야함칰쇼', 0.0],
 [' 사랑을 하고 있어... 지윤이 먹어주면 지영이 대박 해피함', 0.0],
 [' 미첫내...? 방학숙제 많이 없는 사람 존나 해피함', 0.0],
 [' 찾기 어려운것도 있지만 한번 사면 재구매같은걸로 꾸준히 사기도 편하고 빨리 오고 국내에서 구하는것들보다 성분 빵빵하고 싸고 세일 자주함  정도? 쇼핑에 공 좀 들이면 해피쇼핑 가능! 나는 샤워용품도 사서 애용함',
  0.0]]

## 2. 모델 구성

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

from kobert.utils import get_tokenizer 
from kobert.pytorch_kobert import get_pytorch_kobert_model 

from transformers import AdamW 
from transformers.optimization import WarmupLinearSchedule

In [ ]:
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

## 3. 학습

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 2
max_grad_norm = 1 
log_interval = 200 
learning_rate =  5e-5

In [ ]:
# train, validation, test set을 나눠주세요
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(dtls, test_size=0.2, random_state=123)

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = WarmupLinearSchedule(optimizer, warmup_steps=warmup_step, t_total=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad() 
        token_ids = token_ids.long().to(device) 
        segment_ids = segment_ids.long().to(device) 
        valid_length= valid_length 
        label = label.long().to(device) 
        out = model(token_ids, valid_length, segment_ids) 
        loss = loss_fn(out, label) 
        loss.backward() 
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) 
        optimizer.step() 
        scheduler.step()  # Update learning rate schedule 
        train_acc += calc_accuracy(out, label)  
        if batch_id % log_interval == 0: 
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1))) 
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1))) 
    model.eval() #모델 평가 부분 
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc(정확도) {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/261 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1025.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


epoch 1 batch id 1 loss 1.4061554670333862 train acc 0.328125
epoch 1 batch id 201 loss 0.09372039884328842 train acc 0.7631374378109452
epoch 1 train acc 0.8097461685823755


  0%|          | 0/66 [00:00<?, ?it/s]

epoch 1 test acc(정확도) 0.9746685606060606


  0%|          | 0/261 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.07359085232019424 train acc 0.953125
epoch 2 batch id 201 loss 0.08344673365354538 train acc 0.9768345771144279
epoch 2 train acc 0.9777897509578544


  0%|          | 0/66 [00:00<?, ?it/s]

epoch 2 test acc(정확도) 0.9777462121212122


두번 학습 -> 정확도 97% 기록 이대로 저장!

In [ ]:

torch.save(model.state_dict(), 'drive/My Drive//model.pt')
